In [1]:
import numpy as np
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import urllib.request
import time
from sentence_transformers import SentenceTransformer

In [2]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')
train_data.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [3]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

In [4]:
train_data['embedding'] = train_data.apply(lambda row: model.encode(row.Q), axis = 1)

In [24]:
# Create an empty numpy array of given dimensions
embeddings = np.empty((11823, 768))

# Assuming 'train_data' is a dictionary with a key 'embedding' that contains the data you want to insert
# Check if 'embedding' key is present in 'train_data' and if the dimensions match
#if 'embedding' in train_data and np.array(train_data['embedding']).shape == array.shape:
embeddings = np.array(train_data['embedding'].tolist())
#else:
#    print("The shape of train_data['embedding'] doesn't match with the array or 'embedding' key is missing in train_data")


In [25]:
embeddings.shape

(11823, 768)

In [11]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

In [26]:
def return_answer(question):
    t = time.time()
    embedding = model.encode(question)
    train_data['score'] = train_data.apply(lambda x: cos_sim(x['embedding'], embedding), axis=1)
    print('total time: {}'.format(time.time() - t))
    return train_data.loc[train_data['score'].idxmax()]['A']

In [49]:
train_data

,Q,A,label,embedding,score
0,12시 땡!,하루가 또 가네요.,0,"[0.20179579, -0.034438122, 1.5395725, 0.010697...",0.224196
1,1지망 학교 떨어졌어,위로해 드립니다.,0,"[0.07716596, -0.034278188, 0.86244255, 0.02636...",0.182892
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.10445247, -0.012432246, 1.0132878, 0.022501...",0.197974
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0,"[0.09760746, -0.046716798, 0.8936941, 0.021047...",0.190051
4,PPL 심하네,눈살이 찌푸려지죠.,0,"[-0.07002868, 0.031961206, 1.4915429, 4.337195...",0.257890
...,...,...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2,"[0.15363288, -0.3260559, 0.6987679, 0.10914066...",-0.036968
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2,"[0.15363288, -0.3260559, 0.6987679, 0.10914066...",-0.036968
11820,흑기사 해주는 짝남.,설렜겠어요.,2,"[-0.03608513, -0.0052648946, 0.89135355, -0.01...",0.233989
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2,"[-0.08468658, -0.14302948, 0.115261085, 0.3867...",0.100030


In [43]:
return_answer('나랑 커피먹을래?')

total time: 0.6367588043212891


'카페인이 필요한 시간인가 봐요.'

FAISS 적용

In [28]:
import faiss
index = faiss.IndexFlatL2(embeddings.shape[1]) # 초기화 : 벡터의 크기를 지정
index.add(embeddings) # 임베딩을 추가

In [29]:
index.is_trained # 추가가 되었는지 확인하기
index.ntotal # 임베딩 수 확인하기

11823

In [44]:
top_k = 10
query = "나랑 커피먹을래?"
t = time.time()
query_embedding = model.encode([query])
distances, indices = index.search(query_embedding, top_k)
print('total time: {}'.format(time.time() - t))

total time: 0.050106048583984375


In [45]:
indices

array([[4719, 4725, 4727, 4738, 4722, 1016, 1015, 2669, 4336, 4736]])

In [46]:
# 결과 확인
temp = train_data.iloc[indices[0]]
#temp
#temp['distances'] = distances[0]
temp.head(10)

,Q,A,label,embedding,score
4719,커피 마시고 싶어,카페인이 필요한 시간인가 봐요.,0,"[-0.3993267, 0.1972204, 0.624025, 0.0350213, -...",0.833982
4725,커피 좀 마셔야겠다.,좋은 시간 보내시길 바라요.,0,"[-0.61822385, 0.10265435, 0.45700443, 0.170730...",0.821451
4727,커피 중독인가,커피도 좋지만 차도 마셔보세요.,0,"[-0.43330464, 0.18593621, 0.40007284, 0.061037...",0.758893
4738,커피향,커피는 향이죠.,0,"[-0.37359497, 0.050638147, 0.48983014, 0.10150...",0.741882
4722,커피 볶는 시간,커피향 좋겠어요.,0,"[-0.5361387, 0.32619828, 0.92553574, 0.0107541...",0.721663
1016,녹차 한 잔 어때?,드시는거 보기만 해도 좋아요.,0,"[-0.73541343, 0.055900984, -0.042146128, 0.038...",0.725523
1015,녹차 마실래?,드시는거 보기만 해도 좋아요.,0,"[-0.6872571, 0.074429035, 0.11777494, -0.01054...",0.702038
2669,스타벅스 가봤어?,당연하죠.,0,"[0.04279117, 0.60799265, 0.4283862, -0.1353537...",0.735114
4336,진한 커피 한잔,마시면 좋죠.,0,"[-0.23439236, -0.054278757, 0.7673412, 0.19666...",0.646626
4736,커피를 너무 많이 마셔,저도 커피 좋아해요.,0,"[-0.22200698, 0.0684263, 0.43020868, 0.1062221...",0.614607
